In [1]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import optuna

In [2]:
Ridership_df= pd.read_csv('data/Ridership.csv')
df= Ridership_df.copy()
df.sample(5)

,Year,Month,Day,Week Number,Corridor,Workday,Station,Period,Ridership,N_trains,Covid19
34103,2021,February,3,5,Corridor_1,y,Station_1,Midday,32,1,1
31471,2020,November,30,49,Corridor_2,y,Station_5,PM Peak,144,6,1
6139,2019,May,2,18,Corridor_2,y,Station_3,Midday,1102,24,0
57280,2022,July,30,30,Corridor_2,n,Station_5,Weekend/Holiday,10361,37,0
36645,2021,April,14,15,Corridor_4,y,Station_3,Midday,92,6,1


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64369 entries, 0 to 64368
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Year         64369 non-null  int64 
 1   Month        64369 non-null  object
 2   Day          64369 non-null  int64 
 3   Week Number  64369 non-null  int64 
 4   Corridor     64369 non-null  object
 5   Workday      64369 non-null  object
 6   Station      64369 non-null  object
 7   Period       64369 non-null  object
 8   Ridership    64369 non-null  int64 
 9   N_trains     64369 non-null  int64 
 10  Covid19      64369 non-null  int64 
dtypes: int64(6), object(5)
memory usage: 5.4+ MB


In [4]:
df.duplicated().sum()

np.int64(0)

In [5]:
df.isnull().sum()

Year           0
Month          0
Day            0
Week Number    0
Corridor       0
Workday        0
Station        0
Period         0
Ridership      0
N_trains       0
Covid19        0
dtype: int64

In [3]:
def feature_engineering(df):
    # Month mapping
    month_mapping = {
        'January': 1, 'February': 2, 'March': 3, 'April': 4, 'May': 5, 'June': 6,
        'July': 7, 'August': 8, 'September': 9, 'October': 10, 'November': 11, 'December': 12
    }
    df['Month_Num'] = df['Month'].map(month_mapping)

    # Cyclical Day conversion
    def convert_day_to_circle(day):
        angle = 2 * np.pi * (day - 1) / 31
        x = np.cos(angle)
        y = np.sin(angle)
        return x, y

    df['day_x'], df['day_y'] = zip(*df['Day'].map(convert_day_to_circle))

    # Cyclical Week conversion
    def convert_week_to_circle(week):
        angle = 2 * np.pi * (week - 1) / 53
        x = np.cos(angle)
        y = np.sin(angle)
        return x, y

    df['week_x'], df['week_y'] = zip(*df['Week Number'].map(convert_week_to_circle))

    # Categorical features for OneHotEncoder
    categorical_features = ['Month', 'Corridor', 'Workday', 'Station', 'Period']

    # Column transformer for preprocessing
    preprocessor = ColumnTransformer(
        transformers=[
            ('', OneHotEncoder(), categorical_features),
        ], remainder='passthrough'
    )

    # Fit and transform the DataFrame
    df_transformed = preprocessor.fit_transform(df)

    # Convert the encoded features to a dense matrix
    df_transformed = df_transformed.toarray()

    # Convert to DataFrame and get feature names
    feature_names_out = list(preprocessor.get_feature_names_out())
    
    # Convert the dense matrix to a DataFrame
    df_transformed = pd.DataFrame(df_transformed, columns=[item.split('__')[1] for item in feature_names_out])

    return df_transformed

In [4]:
df= feature_engineering(df)

In [5]:
df['COVID_Workday'] = df['Covid19'] * df['Workday_y']

columns_to_convert = [
    'Month_April', 'Month_August', 'Month_December', 'Month_February', 
    'Month_January', 'Month_July', 'Month_June', 'Month_March', 
    'Month_May', 'Month_November', 'Month_October', 'Month_September',
    'Corridor_Corridor_1', 'Corridor_Corridor_2', 'Corridor_Corridor_3', 
    'Corridor_Corridor_4', 'Corridor_Corridor_5', 'Corridor_Corridor_6', 'Corridor_Corridor_7',
    'Station_Station_1', 'Station_Station_10', 'Station_Station_11', 'Station_Station_12', 
    'Station_Station_13', 'Station_Station_14', 'Station_Station_15', 'Station_Station_16', 
    'Station_Station_17', 'Station_Station_18', 'Station_Station_19', 'Station_Station_2', 
    'Station_Station_20', 'Station_Station_21', 'Station_Station_22', 'Station_Station_23', 
    'Station_Station_24', 'Station_Station_25', 'Station_Station_26', 'Station_Station_27', 
    'Station_Station_28', 'Station_Station_29', 'Station_Station_3', 'Station_Station_30', 
    'Station_Station_31', 'Station_Station_32', 'Station_Station_33', 'Station_Station_34', 
    'Station_Station_35', 'Station_Station_36', 'Station_Station_37', 'Station_Station_38', 
    'Station_Station_39', 'Station_Station_4', 'Station_Station_40', 'Station_Station_41', 
    'Station_Station_42', 'Station_Station_43', 'Station_Station_44', 'Station_Station_45', 
    'Station_Station_5', 'Station_Station_6', 'Station_Station_7', 'Station_Station_8', 'Station_Station_9',
    'Period_AM Peak', 'Period_Evening', 'Period_Midday', 'Period_PM Peak', 'Period_Weekend/Holiday',
    'Day',
    'Week Number',
    'Workday_n', 'Workday_y',
    'Covid19','COVID_Workday',
    'Month_Num']

columns_to_convert_02=['Year', 'Ridership', 'N_trains']


df[columns_to_convert] = df[columns_to_convert].astype('uint8')
df[columns_to_convert_02] = df[columns_to_convert_02].astype('uint16')

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64369 entries, 0 to 64368
Data columns (total 83 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Month_April             64369 non-null  uint8  
 1   Month_August            64369 non-null  uint8  
 2   Month_December          64369 non-null  uint8  
 3   Month_February          64369 non-null  uint8  
 4   Month_January           64369 non-null  uint8  
 5   Month_July              64369 non-null  uint8  
 6   Month_June              64369 non-null  uint8  
 7   Month_March             64369 non-null  uint8  
 8   Month_May               64369 non-null  uint8  
 9   Month_November          64369 non-null  uint8  
 10  Month_October           64369 non-null  uint8  
 11  Month_September         64369 non-null  uint8  
 12  Corridor_Corridor_1     64369 non-null  uint8  
 13  Corridor_Corridor_2     64369 non-null  uint8  
 14  Corridor_Corridor_3     64369 non-null

## Problem One: 
Assume we want to find a general model for the number of passengers without looking at the number of passengers on previous days and only using the recorded information in the table for the same time period (except for the year and number of trains) to predict the number of required trains based on the number of passengers.

# spliting

In [6]:
# X_p01 = df.drop(['Ridership', 'Year', 'N_trains'], axis=1)  
# y_p01 = pd.DataFrame(df['Ridership'])

# df_for_stratify_p01 = Ridership_df.copy()
# stratify_key_p01 = df_for_stratify_p01['Covid19'].astype(str) + '_' + \
#                    df_for_stratify_p01['Workday'] + '_' + \
#                    df_for_stratify_p01['Period'] + '_' + \
#                    df_for_stratify_p01['Corridor']

# X_train_p01, X_test_p01, y_train_p01, y_test_p01 = train_test_split(
#     X_p01, y_p01,
#     test_size=0.2,
#     random_state=42,
#     stratify=stratify_key_p01 
# )

In [7]:
def preprocess_and_split_data_p01(df, target_column='Ridership', test_size=0.2, random_state=42):
    # Drop unnecessary columns
    X = df.drop([target_column, 'Year', 'N_trains'], axis=1)
    y = df[target_column]

    # Create stratification key directly from df
    df_for_stratify_p01 = Ridership_df.copy()
    stratify_key_p01 = df_for_stratify_p01['Covid19'].astype(str) + '_' + \
                       df_for_stratify_p01['Workday'] + '_' + \
                       df_for_stratify_p01['Period']+ '_' + \
                       df_for_stratify_p01['Corridor']

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(
        X, y,
        test_size=test_size,
        random_state=random_state,
        stratify=stratify_key_p01
    )

    return X_train, X_test, y_train, y_test

In [8]:
# Example usage:
# Assuming df is already defined and contains the necessary columns
X_train_p01, X_test_p01, y_train_p01, y_test_p01 = preprocess_and_split_data_p01(df)

In [9]:
# train_data = lgb.Dataset(X_train_p01, label=y_train_p01)
# test_data = lgb.Dataset(X_test_p01, label=y_test_p01, reference=train_data)

# # Define parameters
# params = {
#     'objective': 'regression',
#     'metric': 'rmse',
#     'boosting_type': 'gbdt',
#     'num_leaves': 31,
#     'learning_rate': 0.1,
#     'feature_fraction': 0.9,
#     'verbose': -1
# }

# # Train the model
# model_p01 = lgb.train(params,
#                  train_data,
#                  num_boost_round=1000,
#                  valid_sets=[test_data],
#                  callbacks=[lgb.early_stopping(stopping_rounds=50)])

# # Make predictions
# y_pred = model_p01.predict(X_test_p01)

# # Evaluate
# rmse_f_p01 = np.sqrt(mean_squared_error(y_test_p01, y_pred))
# r2_f_p01= r2_score(y_test_p01, y_pred)

# print(f"RMSE: {rmse_f_p01:.2f}")
# print(f"R²: {r2_f_p01:.4f}")

Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[1000]	valid_0's rmse: 825.099
RMSE: 825.10
R²: 0.8129


In [10]:
def objective_p01(trial):
    # Define parameters to be optimized by Optuna
    params = {
        'objective': 'regression',
        'metric': 'rmse',
        'boosting_type': 'gbdt',
        'num_leaves': trial.suggest_int('num_leaves', 20, 100),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 0, 10),
        'lambda_l1': trial.suggest_float('lambda_l1', 0, 10),
        'lambda_l2': trial.suggest_float('lambda_l2', 0, 10),
        'verbose': -1,
        'feature_pre_filter': False # Disable feature pre-filtering
    }


    train_data = lgb.Dataset(X_train_p01, label=y_train_p01)
    test_data = lgb.Dataset(X_test_p01, label=y_test_p01, reference=train_data)

    # Train the model with early stopping on the validation set
    model = lgb.train(params,
                      train_data,
                      num_boost_round=1000, # Max boosting rounds
                      valid_sets=[test_data],
                      callbacks=[lgb.early_stopping(stopping_rounds=50, verbose=False)]) # verbose=False to reduce output

    # Make predictions on the validation set using the best iteration
    y_pred = model.predict(X_test_p01, num_iteration=model.best_iteration)

    # Return RMSE as the objective value to minimize
    return np.sqrt(mean_squared_error(y_test_p01, y_pred))

In [11]:
# Create a study and optimize
study_p01 = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=42)) # Add seed for reproducibility
study_p01.optimize(objective_p01, n_trials=50, show_progress_bar=True) # show_progress_bar for better UX

# Best parameters and best RMSE found by Optuna
best_params_p01 = study_p01.best_params
best_rmse_optuna_p01 = study_p01.best_value
print("\nBest parameters found by Optuna:", best_params_p01)
print(f"Best RMSE on validation set during tuning (Optuna): {best_rmse_optuna_p01:.2f}")

[I 2025-07-21 20:30:00,316] A new study created in memory with name: no-name-a1231877-b820-4676-ae49-8eec62c694df


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-07-21 20:30:03,232] Trial 0 finished with value: 858.3986297975716 and parameters: {'num_leaves': 50, 'learning_rate': 0.28570714885887566, 'min_child_samples': 75, 'max_depth': 8, 'feature_fraction': 0.5780093202212182, 'bagging_fraction': 0.5779972601681014, 'bagging_freq': 0, 'lambda_l1': 8.661761457749352, 'lambda_l2': 6.011150117432088}. Best is trial 0 with value: 858.3986297975716.
[I 2025-07-21 20:30:08,173] Trial 1 finished with value: 965.171480859766 and parameters: {'num_leaves': 77, 'learning_rate': 0.01596950334578271, 'min_child_samples': 98, 'max_depth': 11, 'feature_fraction': 0.6061695553391381, 'bagging_fraction': 0.5909124836035503, 'bagging_freq': 2, 'lambda_l1': 3.0424224295953772, 'lambda_l2': 5.247564316322379}. Best is trial 0 with value: 858.3986297975716.
[I 2025-07-21 20:30:10,319] Trial 2 finished with value: 1000.1536696220307 and parameters: {'num_leaves': 54, 'learning_rate': 0.09445645065743215, 'min_child_samples': 63, 'max_depth': 4, 'feature_

In [12]:
final_model_p01 = lgb.train(best_params_p01,
                            lgb.Dataset(X_train_p01, label=y_train_p01),
                            num_boost_round=1000) # Use best iteration from Optuna's best trial

# Make predictions on the truly unseen test set
y_pred_final_p01 = final_model_p01.predict(X_test_p01)

# Evaluate the final model's performance on the test set
rmse_final_p01 = np.sqrt(mean_squared_error(y_test_p01, y_pred_final_p01))
r2_final_p01 = r2_score(y_test_p01, y_pred_final_p01)

print(f"\nFinal Model Performance on X_test_p01:")
print(f"RMSE: {rmse_final_p01:.2f}")
print(f"R²: {r2_final_p01:.4f}")


Final Model Performance on X_test_p02:
RMSE: 818.90
R²: 0.8157


In [31]:
def calculate_needed_trains(predicted_passengers_series: pd.Series,
                            train_capacity: int = 600,
                            safety_buffer_percentage: float = 0.05,
                            minimum_trains_required: int = 1) -> pd.DataFrame:

    predicted_passengers_int = predicted_passengers_series.astype(int)

    # Create a DataFrame to hold the results
    result_df = pd.DataFrame({
        'Predicted_Passengers': predicted_passengers_int
    })

    # 1. Calculate the base number of trains needed (minimum required)
    # np.ceil ensures that any fraction of a train needed results in a full additional train.
    result_df['Base_Predicted_Trains'] = np.ceil(
        result_df['Predicted_Passengers'] / train_capacity
    ).astype(int)

    # 2. Calculate trains with a safety/comfort buffer
    # This adds a percentage buffer to the predicted passengers before dividing by capacity.
    result_df['Buffered_Predicted_Trains'] = np.ceil(
        result_df['Predicted_Passengers'] * (1 + safety_buffer_percentage) / train_capacity
    ).astype(int)

    # 3. Ensure a minimum number of trains
    # This ensures that even for very low passenger counts, a base service level is maintained.
    result_df['Final_Predicted_Trains'] = result_df['Buffered_Predicted_Trains'].apply(
        lambda x: max(x, minimum_trains_required)
    )

    return result_df

In [36]:
y_pred_train_p01= calculate_needed_trains(y_pred_final_p01)
y_pred_train_p01.sample(10)

,Predicted_Passengers,Base_Predicted_Trains,Buffered_Predicted_Trains,Final_Predicted_Trains
11777,99,1,1,1
1539,2739,5,5,5
4365,565,1,1,1
1191,576,1,2,2
11105,191,1,1,1
51,612,2,2,2
2275,5202,9,10,10
5985,204,1,1,1
6473,17,1,1,1
11604,125,1,1,1


# Problem Two:
 Assume we need to forecast the number of passengers for different time periods in order to allocate the appropriate number of trains one week in advance. (In this case, we are allowed to use information from previous time periods, but using the year and number of trains columns is still not permitted.)

In [17]:
def preprocess_and_split_data_p02(df, target_column='Ridership', train_size=0.8):
    # Convert Year, Month_Num, and Day to a datetime object
    df['Date'] = pd.to_datetime(
        df['Year'].astype(str) + '-' +
        df['Month_Num'].astype(str) + '-' +
        df['Day'].astype(str),
        errors='coerce'  # Handle potential invalid dates by setting them to NaT
    )

    # Drop rows with invalid dates
    df.dropna(subset=['Date'], inplace=True)

    # Grouping columns for creating lagged features
    grouping_cols = [
        col for col in df.columns if col.startswith('Corridor_') or
        col.startswith('Station_') or
        col.startswith('Period_')
    ]

    # Create lagged features for 'Passengers'
    # Lag 7 days (for same day of week, same period, etc., one week prior)
    df['Ridership_Lag_7'] = df.groupby(grouping_cols)['Ridership'].shift(7)

    # Lag 14 days (for same day of week, same period, two weeks prior)
    df['Ridership_Lag_14'] = df.groupby(grouping_cols)['Ridership'].shift(14)

    # Drop rows with NaN values in lagged features
    df.dropna(subset=['Ridership_Lag_7', 'Ridership_Lag_14'], inplace=True)
    
    # Reset index after dropping NaNs
    df.reset_index(drop=True, inplace=True)

    # Sort by Date
    df.sort_values(by=['Date'], inplace=True)

    # Define features (X) and target (y)
    X = df.drop(columns=[target_column, 'Year', 'N_trains', 'Date'])
    y = df[target_column]

    # Calculate split point
    split_point = int(len(df) * train_size)

    # Split the data into training and testing sets
    X_train = X.iloc[:split_point]
    y_train = y.iloc[:split_point]
    
    X_test = X.iloc[split_point:]
    y_test = y.iloc[split_point:]

    # Print shapes of the resulting datasets
    print(f"\nTrain set features (X_train) shape: {X_train.shape}")
    print(f"Test set features (X_test) shape: {X_test.shape}")
    print(f"Train set target (y_train) shape: {y_train.shape}")
    print(f"Test set target (y_test) shape: {y_test.shape}")

    return X_train, X_test, y_train, y_test


In [18]:
X_train_p02, X_test_p02, y_train_p02, y_test_p02 = preprocess_and_split_data_p02(df)


Train set features (X_train) shape: (49951, 82)
Test set features (X_test) shape: (12488, 82)
Train set target (y_train) shape: (49951,)
Test set target (y_test) shape: (12488,)


In [19]:
def objective_p02(trial):
    # Define parameters to be optimized by Optuna
    params = {
        'objective': 'regression',
        'metric': 'rmse',
        'boosting_type': 'gbdt',
        'num_leaves': trial.suggest_int('num_leaves', 20, 100),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 0, 10),
        'lambda_l1': trial.suggest_float('lambda_l1', 0, 10),
        'lambda_l2': trial.suggest_float('lambda_l2', 0, 10),
        'verbose': -1,
        'feature_pre_filter': False # Disable feature pre-filtering
    }

    # Convert data to LightGBM Dataset format
    # Use X_train_p02 for training, and X_test_p02 as the validation set
    # This ensures that Optuna's reported best value is a good generalization estimate.
    train_data = lgb.Dataset(X_train_p02, label=y_train_p02)
    test_data = lgb.Dataset(X_test_p02, label=y_test_p02, reference=train_data)

    # Train the model with early stopping on the validation set
    model = lgb.train(params,
                      train_data,
                      num_boost_round=1000, # Max boosting rounds
                      valid_sets=[test_data],
                      callbacks=[lgb.early_stopping(stopping_rounds=50, verbose=False)]) # verbose=False to reduce output

    # Make predictions on the validation set using the best iteration
    y_pred = model.predict(X_test_p02, num_iteration=model.best_iteration)

    # Return RMSE as the objective value to minimize
    return np.sqrt(mean_squared_error(y_test_p02, y_pred))

In [27]:
# Create a study and optimize
study_p02 = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=42)) # Add seed for reproducibility
study_p02.optimize(objective_p02, n_trials=70, show_progress_bar=True) # show_progress_bar for better UX

# Best parameters and best RMSE found by Optuna
best_params_p02 = study_p02.best_params
best_rmse_optuna_p02 = study_p02.best_value
print("\nBest parameters found by Optuna:", best_params_p02)
print(f"Best RMSE on validation set during tuning (Optuna): {best_rmse_optuna_p02:.2f}")

[I 2025-07-21 20:47:28,545] A new study created in memory with name: no-name-4e6b2d7e-fc20-49ea-a378-c16cce24d001


  0%|          | 0/70 [00:00<?, ?it/s]

[I 2025-07-21 20:47:29,287] Trial 0 finished with value: 749.983216692356 and parameters: {'num_leaves': 50, 'learning_rate': 0.28570714885887566, 'min_child_samples': 75, 'max_depth': 8, 'feature_fraction': 0.5780093202212182, 'bagging_fraction': 0.5779972601681014, 'bagging_freq': 0, 'lambda_l1': 8.661761457749352, 'lambda_l2': 6.011150117432088}. Best is trial 0 with value: 749.983216692356.
[I 2025-07-21 20:47:33,418] Trial 1 finished with value: 689.8169979283266 and parameters: {'num_leaves': 77, 'learning_rate': 0.01596950334578271, 'min_child_samples': 98, 'max_depth': 11, 'feature_fraction': 0.6061695553391381, 'bagging_fraction': 0.5909124836035503, 'bagging_freq': 2, 'lambda_l1': 3.0424224295953772, 'lambda_l2': 5.247564316322379}. Best is trial 1 with value: 689.8169979283266.
[I 2025-07-21 20:47:34,075] Trial 2 finished with value: 669.6805520970248 and parameters: {'num_leaves': 54, 'learning_rate': 0.09445645065743215, 'min_child_samples': 63, 'max_depth': 4, 'feature_fr

In [28]:
final_model_p02 = lgb.train(best_params_p02,
                            lgb.Dataset(X_train_p02, label=y_train_p02),
                            num_boost_round=1000) # Use best iteration from Optuna's best trial

# Make predictions on the truly unseen test set
y_pred_final_p02 = final_model_p02.predict(X_test_p02)

# Evaluate the final model's performance on the test set
rmse_final_p02 = np.sqrt(mean_squared_error(y_test_p02, y_pred_final_p02))
r2_final_p02 = r2_score(y_test_p02, y_pred_final_p02)

print(f"\nFinal Model Performance on X_test_p02:")
print(f"RMSE: {rmse_final_p02:.2f}")
print(f"R²: {r2_final_p02:.4f}")


Final Model Performance on X_test_p02:
RMSE: 632.22
R²: 0.8054


In [29]:
y_pred_final_p02

array([1627.96450871,  360.86406451,  581.99106058, ...,  171.21415554,
       2836.89274428,  537.29573117])

In [32]:
train_allocation_p02= calculate_needed_trains(y_pred_final_p02)

In [34]:
train_allocation_p02.sample(5)

,Predicted_Passengers,Base_Predicted_Trains,Buffered_Predicted_Trains,Final_Predicted_Trains
2987,488,1,1,1
2678,62,1,1,1
8067,938,2,2,2
9587,318,1,1,1
1551,3404,6,6,6


# Problem Three:
 Assume we need to forecast the number of passengers for different time periods in order to allocate a more accurate number of trains for the next day. (In this case, we are allowed to use information from previous time periods, but using the year and number of trains columns is still not permitted.)


In [10]:
def preprocess_and_split_data_p03(df, target_column='Ridership', cut_off_date: str = '2022-12-31', train_size=0.8):
       
    df['Date'] = pd.to_datetime(
    df['Year'].astype(str) + '-' +
    df['Month_Num'].astype(str) + '-' +
    df['Day'].astype(str),
    errors='coerce')

    df.dropna(subset=['Date'], inplace=True)
    df.sort_values(by=['Date'], inplace=True)
    df.reset_index(drop=True, inplace=True)

    # Identify columns that define a unique time-series for lagging
    # These are the columns that define "which" time series we are lagging.
    grouping_cols = [
      col for col in df.columns if 
      col.startswith('Corridor_') or
      col.startswith('Station_') or
      col.startswith('Period_')]
    
    if 'Workday_y' in df.columns:
       grouping_cols.append('Workday_y')
 
    if 'Workday_n' in df.columns:
       grouping_cols.append('Workday_n')

    if 'Covid19' in df.columns:
       grouping_cols.append('Covid19')

    # Create lagged features for 'Passengers'
    # Lag 1 day (most important for next-day forecast)
    df['Ridership_Lag_1'] = df.groupby(grouping_cols)['Ridership'].shift(1)
    # Lag 2 days
    df['Ridership_Lag_2'] = df.groupby(grouping_cols)['Ridership'].shift(2)
    # Lag 7 days (for weekly seasonality, still important)
    df['Ridership_Lag_7'] = df.groupby(grouping_cols)['Ridership'].shift(7)

    # Rolling Average (e.g., 3-day rolling mean, shifted to avoid leakage)
    # min_periods=1 allows calculation even if fewer than 3 periods are available at the start.
    df['Rolling_Avg_Ridership_3'] = df.groupby(grouping_cols)['Ridership'].transform(
       lambda x: x.rolling(window=3, min_periods=1).mean().shift(1))
    
    # Rolling Standard Deviation (e.g., 3-day rolling std, shifted)
    df['Rolling_Std_Ridership_3'] = df.groupby(grouping_cols)['Ridership'].transform(
         lambda x: x.rolling(window=3, min_periods=1).std().shift(1)).fillna(0)

    df.dropna(subset=['Ridership_Lag_1','Ridership_Lag_2','Ridership_Lag_7','Rolling_Avg_Ridership_3'], inplace=True)
    df.reset_index(drop=True, inplace=True)

    # Define features (X) and target (y)
    X = df.drop(columns=[target_column, 'Year', 'N_trains', 'Date'])
    y = df[target_column]

   #  cut_off_datetime = pd.to_datetime(cut_off_date)

   #  train_mask = df['Date'] <= cut_off_datetime
   #  X_train = X[train_mask]
   #  y_train = y[train_mask]

   #  test_mask = df['Date'] > cut_off_datetime
   #  X_test = X[test_mask]
   #  y_test = y[test_mask]

    # Calculate split point
    split_point = int(len(df) * train_size)

    # Split the data into training and testing sets
    X_train = X.iloc[:split_point]
    y_train = y.iloc[:split_point]
    
    X_test = X.iloc[split_point:]
    y_test = y.iloc[split_point:]

    # Print shapes of the resulting datasets
    print(f"\nTrain set features (X_train) shape: {X_train.shape}")
    print(f"Test set features (X_test) shape: {X_test.shape}")
    print(f"Train set target (y_train) shape: {y_train.shape}")
    print(f"Test set target (y_test) shape: {y_test.shape}")

    return X_train, X_test, y_train, y_test

In [11]:
X_train_p03, X_test_p03, y_train_p03, y_test_p03 = preprocess_and_split_data_p03(df)


Train set features (X_train) shape: (47702, 85)
Test set features (X_test) shape: (11926, 85)
Train set target (y_train) shape: (47702,)
Test set target (y_test) shape: (11926,)


In [12]:
def objective_p03(trial):
    # Define parameters to be optimized by Optuna
    params = {
        'objective': 'regression',
        'metric': 'rmse',
        'boosting_type': 'gbdt',
        'num_leaves': trial.suggest_int('num_leaves', 20, 100),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 0, 10),
        'lambda_l1': trial.suggest_float('lambda_l1', 0, 10),
        'lambda_l2': trial.suggest_float('lambda_l2', 0, 10),
        'verbose': -1,
        'feature_pre_filter': False # Disable feature pre-filtering
    }

    # Convert data to LightGBM Dataset format
    # Use X_train_p02 for training, and X_test_p02 as the validation set
    # This ensures that Optuna's reported best value is a good generalization estimate.
    train_data = lgb.Dataset(X_train_p03, label=y_train_p03)
    test_data = lgb.Dataset(X_test_p03, label=y_test_p03, reference=train_data)

    # Train the model with early stopping on the validation set
    model = lgb.train(params,
                      train_data,
                      num_boost_round=1000, # Max boosting rounds
                      valid_sets=[test_data],
                      callbacks=[lgb.early_stopping(stopping_rounds=50, verbose=False)]) # verbose=False to reduce output

    # Make predictions on the validation set using the best iteration
    y_pred = model.predict(X_test_p03, num_iteration=model.best_iteration)

    # Return RMSE as the objective value to minimize
    return np.sqrt(mean_squared_error(y_test_p03, y_pred))

In [15]:
# Create a study and optimize
study_p03 = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=42)) # Add seed for reproducibility
study_p03.optimize(objective_p03, n_trials=70, show_progress_bar=True) # show_progress_bar for better UX

# Best parameters and best RMSE found by Optuna
best_params_p03 = study_p03.best_params
best_rmse_optuna_p03 = study_p03.best_value
print("\nBest parameters found by Optuna:", best_params_p03)
print(f"Best RMSE on validation set during tuning (Optuna): {best_rmse_optuna_p03:.2f}")

[I 2025-07-22 02:25:48,794] A new study created in memory with name: no-name-807a9491-b160-4e12-af4c-02e2b7a36e47


  0%|          | 0/70 [00:00<?, ?it/s]

[I 2025-07-22 02:25:49,086] Trial 0 finished with value: 616.4344150939429 and parameters: {'num_leaves': 50, 'learning_rate': 0.28570714885887566, 'min_child_samples': 75, 'max_depth': 8, 'feature_fraction': 0.5780093202212182, 'bagging_fraction': 0.5779972601681014, 'bagging_freq': 0, 'lambda_l1': 8.661761457749352, 'lambda_l2': 6.011150117432088}. Best is trial 0 with value: 616.4344150939429.
[I 2025-07-22 02:25:50,645] Trial 1 finished with value: 610.784185327549 and parameters: {'num_leaves': 77, 'learning_rate': 0.01596950334578271, 'min_child_samples': 98, 'max_depth': 11, 'feature_fraction': 0.6061695553391381, 'bagging_fraction': 0.5909124836035503, 'bagging_freq': 2, 'lambda_l1': 3.0424224295953772, 'lambda_l2': 5.247564316322379}. Best is trial 1 with value: 610.784185327549.
[I 2025-07-22 02:25:51,528] Trial 2 finished with value: 592.9699728489815 and parameters: {'num_leaves': 54, 'learning_rate': 0.09445645065743215, 'min_child_samples': 63, 'max_depth': 4, 'feature_fr

In [19]:
final_model_p03 = lgb.train(best_params_p03,
                            lgb.Dataset(X_train_p03, label=y_train_p03),
                            num_boost_round=1000) 

# Make predictions on the truly unseen test set
y_pred_final_p03 = final_model_p03.predict(X_test_p03)

# Evaluate the final model's performance on the test set
rmse_final_p03 = np.sqrt(mean_squared_error(y_test_p03, y_pred_final_p03))
r2_final_p03 = r2_score(y_test_p03, y_pred_final_p03)

print(f"\nFinal Model Performance on X_test_p03:")
print(f"RMSE: {rmse_final_p03:.2f}")
print(f"R²: {r2_final_p03:.4f}")


Final Model Performance on X_test_p03:
RMSE: 592.89
R²: 0.8324
